# UMAP

In [1]:
import sys
import seaborn as sns
import pandas as pd 
import numpy as np
from itertools import combinations
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import torch
import anndata as ad
import scanpy as sc
import os
import gc
from importlib import reload
print("C1")

from datasets import Dataset, load_from_disk
from datasets import load_dataset
from geneformer import EmbExtractor

from sklearn.cluster import KMeans
print("C2")

#classifer tools
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
print("C3")

sns.set_style('white')
torch.cuda.empty_cache()

import os

from datasets import Dataset, load_from_disk, load_dataset
import geneformer

from datetime import datetime
print("C4")
DEFAULT_NAME_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_name_id_dict.pkl"
DEFAULT_TOKEN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/token_dictionary.pkl"
DEFAULT_MEDIAN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_median_dictionary.pkl"

sns.set_style('white')
torch.cuda.empty_cache()
print("C5")

KeyboardInterrupt: 

# Importing the anndata files. 

In [2]:
## Iniitla cells
# Just pick from a random file where they were copied (the first half ish of them have them still in original). Then filter by type

harvest_file_path = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/2024-07-31_03-49-01_job_number_22.h5ad'
harvest_file_adata = ad.read_h5ad(harvest_file_path)
initial_adata = harvest_file_adata[harvest_file_adata.obs['type'] == 'initial']


# get adata up and working
# Ensure hsc_adata is already defined and populated with data

# 1. Check the contents of the original obs
print("Original obs contents:")
print(initial_adata.obs.head())

# 2. Create the backup
initial_adata.uns['obs_backup'] = initial_adata.obs.copy()

# 3. Verify the backup was created correctly
print("Contents of obs_backup after creation:")
print(initial_adata.uns['obs_backup'].head())

# 4. Clear the original obs
initial_adata.obs = pd.DataFrame(index=initial_adata.obs.index)

# 5. Check the contents of the cleared obs and the backup
print("Contents of cleared obs:")
print(initial_adata.obs.head())

print("Contents of obs_backup after clearing original obs:")
print(initial_adata.uns['obs_backup'].head())



#initial_adata.obs['kmeans_cluster'] = initial_kmeans.labels_
initial_adata.obs['recipe_name'] = np.nan
initial_adata.obs['recipe_number'] = np.nan
initial_adata.obs['cell_id'] = initial_adata.uns['obs_backup']['cell_id']
initial_adata.obs['type'] = 'initial'



initial_adata.obs.sample(10)



Original obs contents:
                                   input_ids   cell_type         dataset  \
0  [16345  9009 13048 ...  9332 13451  5456]  fibroblast  TS_Vasculature   
1  [12119  9190 16876 ... 11232  1132  1022]  fibroblast  TS_Vasculature   
2  [ 3878  9009  4115 ...  4697 10362 12098]  fibroblast  TS_Vasculature   
3  [ 3878 16916 18367 ... 15470  3946  2153]  fibroblast  TS_Vasculature   
4  [ 6196 16916 10920 ...  7629 10148 17125]  fibroblast  TS_Vasculature   

  length      ignore standardized_cell_type  broad_type __index_level_0__  \
0   2048  fibroblast             Fibroblast  fibroblast            109770   
1   2048  fibroblast             Fibroblast  fibroblast            109771   
2   2048  fibroblast             Fibroblast  fibroblast            109774   
3   2048  fibroblast             Fibroblast  fibroblast            109776   
4   2048  fibroblast             Fibroblast  fibroblast            109777   

  cell_id recipe     type  
0  cell_1    raw  initial  
1

/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_1395202/1375448826.py:17: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  initial_adata.uns['obs_backup'] = initial_adata.obs.copy()


,recipe_name,recipe_number,cell_id,type
6846,NaN,NaN,cell_6847,initial
8572,NaN,NaN,cell_8573,initial
3880,NaN,NaN,cell_3881,initial
12469,NaN,NaN,cell_12470,initial
2547,NaN,NaN,cell_2548,initial
7797,NaN,NaN,cell_7798,initial
7157,NaN,NaN,cell_7158,initial
10699,NaN,NaN,cell_10700,initial
13635,NaN,NaN,cell_13636,initial
14123,NaN,NaN,cell_14124,initial


# ISP Cells

In [3]:
# file = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/' + '2024-07-31_03-17-04_job_number_12.h5ad'
# reprog_centroid_adata_wid = ad.read_h5ad(reprog_wid_path)
# # this is not actually written into the file idk if we care? 
# reprog_centroid_adata_wid.obs['type'] = 'reprogrammed'
# reprog_centroid_adata_wid.obs['cluster_number'] = range(1, len(reprog_centroid_adata_wid.obs) + 1)
# kmeans = KMeans(n_clusters=10, random_state=0)

# # Fit KMeans to the DataFrame (make sure to use the feature columns only)
# reprog_centroid_adata['kmeans'] = kmeans.fit_predict(reprog_centroid_adata)

# reprog_centroid_adata_wid.obs.sample(10)


In [4]:
# from itertools import combinations
# import pandas as pd
# import anndata as ad
# import numpy as np
# from sklearn.cluster import KMeans

# # Initialize an empty AnnData object (This will be replaced by actual data)
# reprog_centroid_adata = ad.AnnData()

# all_cluster_labels = []
# all_cell_id_labels = []
# all_recipe_name_labels = []
# all_recipe_num_labels = []

# def ten_choose_five():
#     gene_list = [
#         'GATA2', 
#         'GFI1B', 
#         'FOS', 
#         'STAT5A',
#         'REL',
#         'FOSB',
#         'IKZF1',
#         'RUNX3',
#         'MEF2C',
#         'ETV6',
#     ]
#     len_sublist = 5
#     sublists = list(combinations(gene_list, len_sublist))
    
#     df = pd.DataFrame({
#         'recipe_iteration': range(1, len(sublists) + 1),
#         'recipe_list': [list(sublist) for sublist in sublists]
#     })
#     return df

# pert_df = ten_choose_five()
# pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
# pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# print("Going to read in ")
# print(file)
# one_recipe_adata = ad.read_h5ad(file)
# print(one_recipe_adata.obs.columns)

# one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

# # Perform k-means clustering
# kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
# one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

# # Collect data for the AnnData object
# all_cell_id_labels = one_recipe_adata.obs['cell_id'].tolist()
# all_cluster_labels = one_recipe_adata.obs['kmeans_clusters'].tolist()

# recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
# if len(recipe_names_this_file) == 1:
#     recipe_as_string = recipe_names_this_file[0]
#     recipe_as_list = recipe_as_string.split(';')
#     print(f'The unique value is: {recipe_as_list}')
#     recipe_num = pert_dict[recipe_as_string]
#     print(recipe_num)
# else:
#     print('Error: There are multiple unique values or no values in the column.')

# # Prepare the AnnData object with all cells
# reprog_centroid_adata = ad.AnnData(X=one_recipe_adata.X)
# reprog_centroid_adata.obs['kmeans_cluster'] = all_cluster_labels
# reprog_centroid_adata.obs['cell_id'] = all_cell_id_labels
# reprog_centroid_adata.obs['recipe_name'] = [recipe_as_string] * len(all_cluster_labels)
# reprog_centroid_adata.obs['recipe_number'] = [recipe_num] * len(all_cluster_labels)

# print("All cell data and kmeans_cluster values have been added to reprog_centroid_adata.")
# print(reprog_centroid_adata.obs.head(12))


In [5]:
# from itertools import combinations
# import pandas as pd
# import anndata as ad
# import numpy as np
# from sklearn.cluster import KMeans
# import os

# def ten_choose_five():
#     gene_list = [
#         'GATA2', 
#         'GFI1B', 
#         'FOS', 
#         'STAT5A',
#         'REL',
#         'FOSB',
#         'IKZF1',
#         'RUNX3',
#         'MEF2C',
#         'ETV6',
#     ]
#     len_sublist = 5
#     sublists = list(combinations(gene_list, len_sublist))
    
#     df = pd.DataFrame({
#         'recipe_iteration': range(1, len(sublists) + 1),
#         'recipe_list': [list(sublist) for sublist in sublists]
#     })
#     return df

# pert_df = ten_choose_five()
# pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
# pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# # List of files to process
# file_list = ['/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-03-23_job_number_89.h5ad','/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-01-47_job_number_88.h5ad']  # Replace with actual file names

# # Initialize a list to collect data
# all_data = []

# for file in file_list:
#     print("Going to read in ", file)
#     one_recipe_adata = ad.read_h5ad(file)
#     print(one_recipe_adata.obs.columns)

#     one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

#     # Perform K-means clustering
#     kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
#     one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

#     # Extract recipe number
#     recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
#     if len(recipe_names_this_file) == 1:
#         recipe_as_string = recipe_names_this_file[0]
#         recipe_num = pert_dict.get(recipe_as_string, None)
#         if recipe_num is None:
#             print(f"Recipe number not found for {recipe_as_string}")
#             continue
#     else:
#         print('Error: There are multiple unique values or no values in the column.')
#         continue

#     # Combine recipe number with kmeans cluster number
#     one_recipe_adata.obs['kmeans'] = one_recipe_adata.obs['kmeans_clusters'].apply(
#         lambda x: int(f"{recipe_num}{x}")
#     )

#     # Collect data
#     all_data.append(one_recipe_adata)

# # Concatenate all AnnData objects
# combined_adata = ad.concat(all_data, join='outer')

# print("Cluster counts:")
# combined_adata.obs.sample(20)

# # she works b"h


In [6]:
# ok here's where we have to be careful.  going back to an old version of the file to find where reprog_centroid_adata_wid was made, and if there's composite of the original cells. checking in turbo first


In [7]:
import os
import re

def list_reprogramming_jobs(directory, start_job_number, end_job_number):
    """
    List and sort files in the given directory by job number within the specified range.

    Parameters:
    directory (str): The directory containing the files.
    start_job_number (int): The starting job number.
    end_job_number (int): The ending job number.

    Returns:
    list: A sorted list of file names within the specified job number range.
    """
    # List all files in the directory
    files = os.listdir(directory)

    # Define a regex pattern to extract job numbers
    pattern = re.compile(r'_job_number_(\d+)\.h5ad$')

    # Create a list of tuples (job_number, file_name)
    files_with_job_numbers = []
    for file in files:
        match = pattern.search(file)
        if match:
            job_number = int(match.group(1))
            if start_job_number <= job_number <= end_job_number:
                files_with_job_numbers.append((job_number, file))

    # Sort files by job number
    files_with_job_numbers.sort(key=lambda x: x[0])

    # Extract sorted file names
    sorted_files = [file for _, file in files_with_job_numbers]

    return sorted_files



directory = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/'
files_in_directory = list_reprogramming_jobs(directory, 1, 252)
files_in_directory = [os.path.join(directory,file) for file in files_in_directory]
files_in_directory

['/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-44_job_number_1.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-45_job_number_2.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-53_job_number_3.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-49_job_number_4.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-45-28_job_number_5.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-55_job_number_6.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-45-23_job_number_7.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-57_job_number_8.h5ad',
 '/nfs/turbo/umm

In [ ]:
from itertools import combinations
import pandas as pd
import anndata as ad
import numpy as np
from sklearn.cluster import KMeans
import os

def ten_choose_five():
    gene_list = [
        'GATA2', 
        'GFI1B', 
        'FOS', 
        'STAT5A',
        'REL',
        'FOSB',
        'IKZF1',
        'RUNX3',
        'MEF2C',
        'ETV6',
    ]
    len_sublist = 5
    sublists = list(combinations(gene_list, len_sublist))
    
    df = pd.DataFrame({
        'recipe_iteration': range(1, len(sublists) + 1),
        'recipe_list': [list(sublist) for sublist in sublists]
    })
    return df

pert_df = ten_choose_five()
pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# List of files to process
  # Replace with actual file names

# Initialize a list to collect data
all_data = []

for file in files_in_directory:
    print("Going to read in ", file)
    one_recipe_adata = ad.read_h5ad(file)
    print(one_recipe_adata.obs.columns)

    one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
    one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

    # Extract recipe number
    recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
    if len(recipe_names_this_file) == 1:
        recipe_as_string = recipe_names_this_file[0]
        recipe_num = pert_dict.get(recipe_as_string, None)
        if recipe_num is None:
            print(f"Recipe number not found for {recipe_as_string}")
            continue
    else:
        print('Error: There are multiple unique values or no values in the column.')
        continue

    # Combine recipe number with kmeans cluster number
    one_recipe_adata.obs['kmeans'] = one_recipe_adata.obs['kmeans_clusters'].apply(
        lambda x: int(f"{recipe_num}{x}")
    )

    # Collect data
    all_data.append(one_recipe_adata)

# Concatenate all AnnData objects
combined_adata = ad.concat(all_data, join='outer')

# she works b"h


Going to read in  /nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-44_job_number_1.h5ad


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['input_ids', 'cell_type', 'dataset', 'length', 'ignore',
       'standardized_cell_type', 'broad_type', '__index_level_0__', 'cell_id',
       'recipe', 'type'],
      dtype='object')


/tmp/ipykernel_1395202/127142444.py:49: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_


Going to read in  /nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-45_job_number_2.h5ad


...

Going to read in  /nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_12-44-03_job_number_208.h5ad


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['input_ids', 'cell_type', 'dataset', 'length', 'ignore',
       'standardized_cell_type', 'broad_type', '__index_level_0__', 'cell_id',
       'recipe', 'type'],
      dtype='object')


/tmp/ipykernel_1395202/127142444.py:49: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_


Going to read in  /nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_12-44-38_job_number_209.h5ad


In [ ]:
# and we pray 
combined_adata.obs.sample(20)

In [ ]:


# youve gotta delete the duplicate 252 again girlpop
# Assuming `combined_adata` is your AnnData object

# Get the rows where 'kmeans_clusters' is equal to 252
rows_with_252 = (combined_adata.obs['kmeans'] >= 2520)

# Count the number of rows with kmeans_clusters equal to 252
num_rows_252 = rows_with_252.sum()

print(f"Number of rows with 'kmeans' equal to 252: {num_rows_252}")


In [ ]:
mask = (combined_adata.obs['kmeans'] >= 1830) | (combined_adata.obs['kmeans'] <= 1839) 
# Filter the DataFrame based on the mask
test_obs = combined_adata.obs[mask]

# Step 2: Count the number of unique rows
num_unique_rows = test_obs.shape[0]
num_unique_rows

In [ ]:
# get rid of all rows with kmeans >= 2520 doesn't solve it. the other dataframe with < 2520 is a whack job. on the one hand,
# it seems like each bin (binned together by recipe) has 15308 in it with the exception of 0-9 bc we started cluster number at 10 
# instead of 00.
# that actually doesnt fix it we conditioned fine


In [ ]:
# import pandas as pd

# # Assuming `combined_adata` is your AnnData object and 'relevant_column' is the column to filter

# # Step 1: Create masks for rows based on the condition
# mask_less_equal = combined_adata.obs['kmeans'] < 2520
# mask_greater = combined_adata.obs['kmeans'] >= 2520

# # Filter the DataFrame based on these masks
# filtered_obs_less_equal = combined_adata.obs[mask_less_equal]
# filtered_obs_greater = combined_adata.obs[mask_greater]

# # Step 2: Remove duplicate rows from those with values greater than 2520
# unique_filtered_obs_greater = filtered_obs_greater.drop_duplicates()

# # Combine the two filtered DataFrames
# combined_filtered_obs = pd.concat([filtered_obs_less_equal, unique_filtered_obs_greater])

# # Step 3: Update `combined_adata` with the combined filtered DataFrame
# combined_adata_no_dupe = combined_adata[combined_adata.obs.index.isin(combined_filtered_obs.index)]

# # Print the number of remaining rows
# print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")
# # 3872924 is (252*15308) + 15309 extra from the extra 252 file.


In [ ]:
# import pandas as pd

# # Assuming `combined_adata` is your AnnData object and 'kmeans' is the column to filter

# # Step 1: Create masks for rows based on the condition
# mask_less_equal = combined_adata.obs['kmeans'] < 2520
# mask_greater = combined_adata.obs['kmeans'] >= 2520

# # Filter the DataFrame based on these masks
# filtered_obs_less_equal = combined_adata.obs[mask_less_equal]
# filtered_obs_greater = combined_adata.obs[mask_greater]

# # Step 2: Remove duplicate rows from those with values greater than 2520
# # Consider dropping duplicates based on all columns
# unique_filtered_obs_greater = filtered_obs_greater.drop_duplicates()

# # Print the number of rows before and after dropping duplicates
# print(f"Number of rows before dropping duplicates: {filtered_obs_greater.shape[0]}")
# print(f"Number of rows after dropping duplicates: {unique_filtered_obs_greater.shape[0]}")

# # Combine the two filtered DataFrames
# combined_filtered_obs = pd.concat([filtered_obs_less_equal, unique_filtered_obs_greater])

# # Step 3: Update `combined_adata` with the combined filtered DataFrame
# combined_adata_no_dupe = combined_adata[combined_adata.obs.index.isin(combined_filtered_obs.index)]

# # Print the number of remaining rows
# print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")

# # Additional check: Print the counts of 'kmeans' values before and after filtering
# print("Counts before filtering:")
# print(combined_adata.obs['kmeans'].value_counts())

# print("Counts after filtering:")
# print(combined_adata_no_dupe.obs['kmeans'].value_counts())


In [ ]:
# import pandas as pd
# import numpy as np

# # Assuming `combined_adata` is your AnnData object and 'kmeans' is the column to filter

# # Step 1: Create masks for rows based on the condition
# filtered_less = combined_adata[combined_adata.obs['kmeans'] < 2520]
# greater_equal = combined_adata[combined_adata.obs['kmeans'] >= 2520]

# # # Filter the DataFrame based on these masks
# # filtered_obs_less_equal = combined_adata.obs[mask_less_equal]
# # filtered_obs_greater = combined_adata.obs[mask_greater]
# print("C1")

# # Step 2: Remove duplicate rows from those with values greater than 2520
# filtered_greater = greater_equal[~greater_equal.obs.duplicated(keep='first')]
# #filtered_greater.X
# filtered_less.X.shape[0]
# #combined_adata_no_dupe = ad.concat([filtered_less, filtered_greater], join='outer')
# # print("C2")
# # # Create a boolean mask to select rows from `combined_adata` that are either:
# # # - Less than 2520 (no duplicates) 
# # # - Greater than or equal to 2520 and are unique
# # # mask_final = np.concatenate([
# # #     mask_less_equal.values,  # All rows where 'kmeans' < 2520
# # #     np.isin(combined_adata.obs.index, unique_filtered_obs_greater.index)  # Unique rows where 'kmeans' >= 2520
# # # ])
# # mask_final = m
# # print("C3")
# # # Step 3: Filter the AnnData object based on the combined mask
# # combined_adata_no_dupe = combined_adata[mask_final]

# # # Print the number of remaining rows
# # print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")
# # print("C4")
# # # Additional check: Print the counts of 'kmeans' values before and after filtering
# # print("Counts before filtering:")
# # print(combined_adata.obs['kmeans'].value_counts())

# # print("Counts after filtering:")
# # print(combined_adata_no_dupe.obs['kmeans'].value_counts())
# # print("C5")

In [ ]:
# import pandas as pd

# # Assuming `filtered_less` is the DataFrame or AnnData object you want to analyze

# # Create bins with ranges of 10
# bins = range(0, filtered_less.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(filtered_less.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Check if any bin does not have exactly 15,308 counts
# target_count = 15308
# bins_with_different_count = binned_counts[binned_counts != target_count]

# # Print bins that do not have exactly 15,308 counts
# print(f"Total number of bins: {len(binned_counts)}")
# if not bins_with_different_count.empty:
#     print("Bins with counts different from 15,308:")
#     print(bins_with_different_count)
# else:
#     print("All bins have exactly 15,308 counts.")


In [ ]:
# import pandas as pd

# # Assuming `filtered_less` is the DataFrame or AnnData object you want to analyze

# # Create bins with ranges of 10
# bins = range(0, filtered_less.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(filtered_less.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Print the count in each bin
# print("Counts in each bin:")
# print(binned_counts)

# # Check if any bin does not have exactly 15,308 counts
# target_count = 15308
# bins_with_different_count = binned_counts[binned_counts != target_count]

# # Print bins that do not have exactly 15,308 counts
# print(f"Total number of bins: {len(binned_counts)}")
# if not bins_with_different_count.empty:
#     print("Bins with counts different from 15,308:")
#     print(bins_with_different_count)
# else:
#     print("All bins have exactly 15,308 counts.")


In [ ]:
# import pandas as pd
# import numpy as np

# # Assuming `combined_adata` is your AnnData object and 'kmeans' is the column to filter

# # Step 1: Create masks for rows based on the condition
# mask_less_equal = combined_adata.obs['kmeans'] < 2520
# mask_greater = combined_adata.obs['kmeans'] >= 2520

# # Filter the DataFrame based on these masks
# filtered_obs_greater = combined_adata.obs[mask_greater]

# # Step 2: Remove duplicate rows from those with values greater than 2520
# unique_filtered_obs_greater = filtered_obs_greater.drop_duplicates()

# # Convert the index of unique_filtered_obs_greater to a boolean mask
# unique_mask = combined_adata.obs.index.isin(unique_filtered_obs_greater.index)

# # Combine the masks for filtering
# final_mask = mask_less_equal | unique_mask

# # Step 3: Filter the AnnData object based on the combined mask
# combined_adata_no_dupe = combined_adata[final_mask]

# # Print the number of remaining rows
# print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")

# # Additional check: Print the counts of 'kmeans' values before and after filtering
# print("Counts before filtering:")
# print(combined_adata.obs['kmeans'].value_counts())

# print("Counts after filtering:")
# print(combined_adata_no_dupe.obs['kmeans'].value_counts())


In [ ]:
# import pandas as pd

# # Assuming `combined_adata` is your AnnData object

# # Create bins with ranges of 10
# bins = range(0, combined_adata_no_dupe.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(combined_adata_no_dupe.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Print the counts for each bin
# print(binned_counts)

# import pandas as pd

# # Assuming `combined_adata` is your AnnData object

# # Create bins with ranges of 10
# bins = range(0, combined_adata_no_dupe.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(combined_adata_no_dupe.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Check if any bin does not have exactly 15,308 counts
# target_count = 15308
# bins_with_different_count = binned_counts[binned_counts != target_count]

# # Print bins that do not have exactly 15,308 counts
# if not bins_with_different_count.empty:
#     print("Bins with counts different from 15,308:")
#     print(bins_with_different_count)
# else:
#     print("All bins have exactly 15,308 counts.")


In [ ]:
# # # Assuming `combined_adata` is your AnnData object

# # # Get the unique values and their counts in the 'kmeans' column
# # kmeans_counts = combined_adata.obs['kmeans'].value_counts()

# # # Print the unique values and their counts
# # print(kmeans_counts)
# # # Assuming `combined_adata` is your AnnData object

# # # Calculate the average value in the 'kmeans' column
# # average_kmeans = combined_adata.obs['kmeans'].mean()

# # # Print the average value
# # print(f"Average value of 'kmeans': {average_kmeans}")
# # Assuming `combined_adata` is your AnnData object

# # Filter for 'kmeans' values between 2520 and 2529
# kmeans_range = combined_adata_.obs['kmeans'][(combined_adata.obs['kmeans'] >= 2520) & (combined_adata.obs['kmeans'] <= 2529)]

# # Get the count of each 'kmeans' value within the range
# kmeans_counts = kmeans_range.value_counts().sort_index()

# # Print the counts for each 'kmeans' value in the range 2520 to 2529
# print(kmeans_counts)
# # sum to 30616 so that's good

In [ ]:
# # writing into turbo
# combined_adata = combined_adata[combined_adata.obs['type'] == 'reprogrammed']
# combined_adata.obs.sample(10)
# combined_adata.write('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/perturbed_combined.h5ad')

# I'm just going to pretend like this isn't an issue for now.

In [ ]:
combined_adata.X.shape

In [ ]:
import pandas as pd
import numpy as np

# Assuming combined_adata is your AnnData object and 'kmeans' is the column to analyze

# Get the minimum and maximum values
min_value = combined_adata.obs['kmeans'].min()
max_value = combined_adata.obs['kmeans'].max()

# Define the bin edges with a width of 10
bins = np.arange(min_value, max_value + 10, 10)

# Define bin labels
bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# Categorize 'kmeans' values into bins
binned_kmeans = pd.cut(combined_adata.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# Count occurrences in each bin
binned_counts = binned_kmeans.value_counts().sort_index()

# Prepare the results as an array of intervals and their counts
intervals_counts = np.array(list(zip(binned_counts.index, binned_counts.values)))

# Print the intervals and counts
print("Intervals and their counts:")
print(intervals_counts)


In [ ]:
import pandas as pd
import numpy as np

# Assuming combined_adata is your AnnData object and 'kmeans' is the column to analyze

# Get the minimum and maximum values
min_value = combined_adata.obs['kmeans'].min()
max_value = combined_adata.obs['kmeans'].max()

# Define the bin edges with a width of 10
bins = np.arange(min_value, max_value + 10, 10)

# Define bin labels
bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# Categorize 'kmeans' values into bins
binned_kmeans = pd.cut(combined_adata.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# Count occurrences in each bin
binned_counts = binned_kmeans.value_counts().sort_index()

# Prepare the results as an array of intervals and their counts
intervals_counts = np.array(list(zip(binned_counts.index, binned_counts.values)))

# Calculate the sum of the second column (counts)
sum_of_counts = intervals_counts[:, 1].astype(int).sum()

# Print the sum of the counts
print(f"Sum of counts in all bins: {sum_of_counts}")


In [ ]:
# separating out into problem part and not problem part:
import pandas as pd

# Define the bin label for the high bin
high_bin_label = '2520-2529'

# Create a mask for rows in the '2520-2529' bin
high_bin_mask = binned_kmeans == high_bin_label

# Create the high and low DataFrames based on the mask
condensed_adata_high = combined_adata[high_bin_mask]
condensed_adata_low = combined_adata[~high_bin_mask]

# Print the number of rows in each DataFrame to verify
print(f"Number of rows in condensed_adata_high: {condensed_adata_high.n_obs}")
print(f"Number of rows in condensed_adata_low: {condensed_adata_low.n_obs}")


In [ ]:
import pandas as pd
import numpy as np
import anndata

def remove_duplicate_obs(adata):
    # Identify duplicate indices
    duplicated_indices = adata.obs.index.duplicated(keep='first')
    
    # Remove duplicate observations
    adata = adata[~duplicated_indices].copy()
    
    return adata

condensed_adata_high = remove_duplicate_obs(condensed_adata_high)
#condensed_adata_high.n_obs
condensed_adata_high

In [ ]:
combined_adata_no_dupe_true = ad.concat([condensed_adata_low, condensed_adata_high])
combined_adata_no_dupe_true.obs['kmeans'] -= 10
combined_adata_no_dupe_true.obs.tail(10)

In [ ]:
# what you see above is what's in the file.
combined_adata_no_dupe_true.write('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/perturbed_combined_good.h5ad')
print("donesies")

# New approach

#### Loading this all in is too hard. let's just take clusters directly from the files we need.

In [ ]:
#'/nfs/turbo/umms-indikar/shared/projects/geneformer/8_14_originals_working/fib15k/2024-07-31_05-34-28_job_number_61.h5ad'
#cluster 7 when 0-9
# pick some rando hsc cells idrc/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/

In [ ]:
# mean comes from any cluster* in file 77